# Notebook 16: Expectation Maximization in practice


## Exercise 1: What if we know everything?
<ul>
<li>Consider first the case where we have complete knowledge of the experiment, namely, both  $\textbf{x}$ and  $\textbf{z}$ are known. How would you intuitively estimate the biases of the two coins $\boldsymbol{\theta}=(\theta_A,\theta_B)$.?</li>
<span style="color:blue">    
    $$
\begin{aligned}    
\theta_A &= \dfrac{\sum_{i=1}^{n=5} (1-z_i)x_i}{10 \sum_{i=1}^{n=5} (1-z_i)} \\
\theta_B &= \dfrac{\sum_{i=1}^{n=5} z_i x_i}{10 \sum_{i=1}^{n=5} z_i} \\
\end{aligned}
$$ 
where $z_i$ is 0 if experiment $i$ used coin $A$ and $1$ if it used coin $B$.
 </span>
<li>What's the likelihood of observing the complete outcomes of these experiments? In other words, what is  $P(\textbf{x},\textbf{z}|n,\boldsymbol{\theta})$ ? You may assume this is a Bernoulli trial. Namely, every time coin $A$ ($B$) is tossed, we have, with probability $\theta_A$ ($\theta_B$), that the outcome is heads.</li>
<span style="color:blue">    
Let $p_i = (1-z_i)\theta_A +z_i\theta_B$. Then
$$
   P(\textbf{x},\textbf{z}|n,\boldsymbol{\theta}) = \dfrac{1}{2^n}\prod_{i=1}^{n} {10 \choose x_i} p_i^{x_i}(1-p_i)^{10-x^i} 
$$
</span>

<li>What's the Maximum Likelihood Estimator (MLE)? Is this consistent with your intuition?</li>
<span style="color:blue"> 
$$
   \dfrac{\partial \log P(\textbf{x},\textbf{z}|n,\boldsymbol{\theta})}{\partial \theta_A} = 0 
   \implies \sum_{i=1}^n (1-z_i)\left[ \dfrac{x_i}{\theta_A} - \dfrac{10-x_i}{1-\theta_A} \right] = 0
   \implies \theta_A = \dfrac{\sum_{i=1}^{n=5} (1-z_i)x_i}{10 \sum_{i=1}^{n=5} (1-z_i)} 
$$    
Similarly for $\theta_B$. We got the same results in the first question.
</span>

</ul>	


In [1]:
import numpy as np
from scipy.special import comb
import math
import time
import matplotlib.pyplot as plt
import random

In [115]:
def compute_likelihood(obs, n, pheads):
    likelihood = comb(n, obs, exact=True)*(pheads**obs)*(1.0-pheads)**(n-obs) 
    #likelihood = (pheads**obs)*(1.0-pheads)**(n-obs) 
    return likelihood

# Generate experiments
num_coin_toss = 100 # each experiment contains num_coin_toss tosses
num_exp = 50  # we perform 5 such experiments
theta_A_true = 0.8 
theta_B_true = 0.4

# MLE 
MLE_A = 0.0
MLE_B = 0.0

print("True values:",(theta_A_true,theta_B_true))

# Generate the outcomes of experiment
coin_choice = np.zeros(num_exp) # initialize: 0 for A and 1 for B
head_counts = np.zeros(num_exp)
for i in np.arange(num_exp):
    
    if np.random.randint(2) == 0: # coin A is selected
        head_counts[i] = np.random.binomial(num_coin_toss , theta_A_true, 1) # toss coin A num_coin_toss times
        MLE_A = MLE_A +  head_counts[i] # add the number of heads observed to total headcounts 
    
    else: # coin B is selected 
        head_counts[i] = np.random.binomial(num_coin_toss , theta_B_true, 1) # toss coin B num_coin_toss times
        coin_choice[i] = 1  # record the selection of coin B during experiment i 
        MLE_B = MLE_B +  head_counts[i] # add the number of heads observed to total headcounts 
tail_counts = num_coin_toss - head_counts

########
def MLE(zv=None):
    if zv is None:
        Z = np.sum(coin_choice)
        X = np.sum(head_counts)
        XZ = np.sum(coin_choice*head_counts)
        return (X-XZ)/(num_coin_toss*(num_exp-Z)), XZ/(num_coin_toss*Z)
    else:
        Z = np.sum(coin_choice)
        X = np.sum(head_counts)
        XZ = np.sum(coin_choice*head_counts)
        return np.sum(zv[0]*head_counts)/(num_coin_toss*np.sum(zv[0])), np.sum(zv[1]*head_counts)/(num_coin_toss*np.sum(zv[1]))
def solve_cubic(a,b,c,d):
    D0 = b**2 - 3*a*c
    D1 = 2*b**3 - 9*a*b*c + 27*a**2*d
    C = ( ( D1 + np.sqrt( D1**2 - 4*D0**3 +0j ) )/2 )**(1/3)
    xi = (-1+1j*np.sqrt(3))/2
    def root(i):
        return -( b + xi**i*C + D0/(xi**i*C) )/(3*a)
    roots = [root(0), root(1), root(2)]
    roots_ = [np.real(i) for i in roots if np.abs(np.imag(i))<1e-6 and np.real(i)<=1 and np.real(i)>=0]
    return max(roots_)
def MAP(zv=None, uA=0.83, sA=1, uB=0.37, sB=1):
    if zv is None:
        nA = np.sum(1-coin_choice)
        nB = np.sum(coin_choice)

        bA = -(1+uA)
        cA = uA - num_coin_toss*nA*sA**2
        dA = sA**2 * np.sum((1-coin_choice)*head_counts)

        bB = -(1+uB)
        cB = uB - num_coin_toss*nB*sB**2
        dB = sB**2 * np.sum(coin_choice*head_counts)
        return solve_cubic(1,bA,cA,dA), solve_cubic(1,bB,cB,dB)
    else:
        nA = np.sum(zv[0])
        nB = np.sum(zv[1])

        bA = -(1+uA)
        cA = uA - num_coin_toss*nA*sA**2
        dA = sA**2 * np.sum(zv[0]*head_counts)

        bB = -(1+uB)
        cB = uB - num_coin_toss*nB*sB**2
        dB = sB**2 * np.sum(zv[1]*head_counts)
        return solve_cubic(1,bA,cA,dA), solve_cubic(1,bB,cB,dB)    
########

start = time.time()
res = MLE()
print("\nMLE:", res)
end = time.time()
print("Time:", end - start)
print("Error:", np.sqrt( (res[0]-theta_A_true)**2 + (res[1]-theta_B_true)**2 ))

start = time.time()
res = MAP()
print("\nMAP:", res)
end = time.time()
print("Time:", end - start)
print("Error:", np.sqrt( (res[0]-theta_A_true)**2 + (res[1]-theta_B_true)**2 ))

# MLE is merely the proportion of heads for each coin toss
MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice))*num_coin_toss)
MLE_B = MLE_B / (np.count_nonzero(coin_choice)*num_coin_toss)

def EM(pA_initial, pB_initial, method=MLE):
    # initialize the pA(heads) and pB(heads), namely, coin biases
    pA_heads = np.zeros(100); 
    pB_heads = np.zeros(100); 

    pA_heads[0] = pA_initial # 0.60 # initial guess
    pB_heads[0] = pB_initial # 0.50 # initial guess

    # E-M begins!
    epsilon = 0.001   # error threshold
    j = 0 # iteration counter
    improvement = float('inf')

    while (improvement > epsilon):

        expectation_A = np.zeros((num_exp,2), dtype=float) 
        expectation_B = np.zeros((num_exp,2), dtype=float)
        weightsA = np.zeros(num_exp, dtype=float)
        weightsB = np.zeros(num_exp, dtype=float)
        
        for i in np.arange(min(len(head_counts),len(tail_counts))):

            eH = head_counts[i]
            eT = tail_counts[i]

            # E step:
            lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
            lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])

            weightA = lA / (lA + lB)
            weightB = lB / (lA + lB)
            
            weightsA[i] = weightA
            weightsB[i] = weightB

            expectation_A[i] = weightA*np.array([eH, eT])
            expectation_B[i] = weightB*np.array([eH, eT])

        # M step
        theta_A = np.sum(expectation_A, axis = 0)[0] / np.sum(expectation_A) 
        theta_B = np.sum(expectation_B, axis = 0)[0] / np.sum(expectation_B) 
        theta_A, theta_B = method(zv=[ weightsA, weightsB ]) # MLE

        #print('At iteration %d, theta_A = %2f,  theta_B = %2f' % (j, theta_A, theta_B))

        pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A)); 
        pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B)); 

        improvement = max( abs(np.array([pA_heads[j+1],pB_heads[j+1]]) - np.array([pA_heads[j],pB_heads[j]]) ))
        j = j+1
    if theta_A>theta_B: return theta_A, theta_B, j
    else: return theta_B, theta_A, j

pA0, pB0 = np.random.random(2)
start = time.time()
res = EM(pA0, pB0, MLE)
end = time.time()
print('\nE-M (without prior):', res[0:2])
print('...converges at iteration %d' %res[-1])
print("Time:", end - start)
print("Error:", np.sqrt( (res[0]-theta_A_true)**2 + (res[1]-theta_B_true)**2 ))

pA0, pB0 = random.gauss(0.83, 1), random.gauss(0.37, 1)
start = time.time()
res = EM(pA0, pB0, MAP)
end = time.time()
print('\nE-M (with prior):', res[0:2])
print('...converges at iteration %d' %res[-1])
print("Time:", end - start)
print("Error:", np.sqrt( (res[0]-theta_A_true)**2 + (res[1]-theta_B_true)**2 ))

True values: (0.8, 0.4)

MLE: (0.802, 0.404)
Time: 0.0
Error: 0.004472135954999584

MAP: (0.8020017783902068, 0.40399672566151273)
Time: 0.0010271072387695312
Error: 0.004470003661843356

E-M (without prior): (0.802000000018688, 0.4040000001135043)
...converges at iteration 3
Time: 0.004998683929443359
Error: 0.004472136064878382

E-M (with prior): (0.8020017784084726, 0.40399672577363166)
...converges at iteration 4
Time: 0.00693821907043457
Error: 0.0044700037702711725


In [88]:
TA = []; TB = []; ERROR=[]; TIME = []; IG = []
start = time.time()    
for i in range(20):
    IG.append( np.random.random(2) )
    start = time.time()   
    _theta_A, _theta_B, _  = EM(*IG[-1])
    end = time.time()
    TA.append(_theta_A); TB.append(_theta_B); TIME.append(end - start)
    ERROR.append( np.sqrt( (_theta_A-theta_A_true)**2 + (_theta_B-theta_B_true)**2 ) )
TA = np.array(TA); TB = np.array(TB)

print("Best time:", min(TIME))
print("Optimal initial guesses for EM:")
for j in [i for i in range(len(ERROR)) if ERROR[i] == min(ERROR)]: print(IG[j], " ---> ", TA[j], TB[j])
print("Min error:", min(ERROR))    

Best time: 0.0
Optimal initial guesses for EM:
[0.82131187 0.38235789]  --->  0.7927018961429952 0.3361422556755914
Min error: 0.06427343020338316


## Exercise 2
<ul>
<li>How fast does EM converge? Is the converged result close to what you'd get from MLE?</li>
<span style="color:blue">    
The performance of the EM algorithm is sensitive to the initial conditions. For instance, when the initial conditions for $\theta_A$ and $\theta_B$ are approximately equal, the algorithm seems to require fewer iterations, but it also produces the worst results. In general, MLE is faster and produces almost the same results as the EM algorithm. In this particular case, the execution time of the EM algorithm can be reduced by removing the binomial coefficients, which are simplified when calculating the weights and, therefore, do not affect the results.
</span>
<li>Following Exercise 1, what's the objective function we're optimizing in the E-step? Does this function have a unique global maximum?</li>
<span style="color:blue">       
We want to maximize the log-likelihood. In the first section, we found the values of $\theta_A$ and $\theta_B$ that maximize this likelihood. The results of the EM algorithm will be a bit different, since the indicators $1-z_i$ and $z_i$ are replaced by  weights $w^{(A)}_i$ and $w^{(B)}_i$. Furthermore, depending on the initial conditions, the EM algorithm can converge to values close to $(\theta_A,\theta_B)$ or to values close to $(\theta_B,\theta_A)$.
</span>
    <li>Compare both the results of MLE and EM to the actual bias (i.e. <code>theta_A_true</code> and <code>theta_B_true</code> in the snippet above), comment on their performance.</li>
<span style="color:blue">    
The results are sensitive to the initial conditions. In general, MLE and the EM algorithm produce similar results.
</span>
    
</ul>

## Exercise 3
<ul>

<li> Now instead of having a fixed initial guess of coin biases (i.e. <code>pA_heads[0]</code> and <code>pB_heads[0]</code> in the snippet), draw these values uniformly at random from $[0,1]$ and run the E-M algorithm. Repeat this twenty times and report what you observed. What's the best initial guess that gives the closest estimate to the true parameters? </li>
<span style="color:blue">    
The worst results are obtained when the initial conditions for $\theta_A$ and $\theta_B$ are approximately equal.
 </span>

<li> As we discussed in Section X (LinReg), Maximum a posteriori (MAP) estimation differs from MLE in that it employs an augmented objective function which incorporates a prior distribution over the quantities we want to estimate, and the prior distribution can be think of as a regularizer for the objective fuction used in MLE. Here we will explore how to extend E-M to MAP estimation. </li>
<ol>
<li> First derive the MAP estimate for the one-coin-flipping example, namely,
$$  \hat{\theta}_\mathrm{MAP}(\textbf{x})=\arg\max_{\theta \in[0,1]}\log P(\textbf{x}|n,\theta)+\log P(\theta), $$
 where
$$ P(\textbf{x}|n,\theta)= \prod_i^{10} \mathrm{Binomial}(x_i|n,\theta) \\
P(\theta) = \mathcal{N}(\theta|\mu,\sigma)
$$  
<span style="color:blue"> 
$$
  \dfrac{\partial \left[ \log P(\textbf{x},\textbf{z}|n,\boldsymbol{\theta}) + \log P(\theta) \right]}{\partial \theta_A} = 0 
   \implies \sum_{i=1}^n (1-z_i)\left[ \dfrac{x_i}{\theta_A} - \dfrac{10-x_i}{1-\theta_A} \right] -\dfrac{\theta_A-\mu_A}{\sigma_A^2} = 0 \\ 
  \implies \theta_A^3 - (1+\mu_A) \theta_A^2 + \left[ \mu_A  -\sigma_A^2 10 \sum_{i=1}^n (1-z_i)\right]  \theta_A+ \sigma_A^2  \sum_{i=1}^n (1-z_i)x_i  = 0
$$ 
This is a cubic equation with exact solutions that are provided in the function <code>solve_cubic</code> defined above.    
</span>  
</li> 
<li> Based on 1., now modify the E-M snippet above to incorporate this prior distribution into the M-step. Comment on the performance. For the prior choice, try $P(\theta)=\mathcal{N}(\theta_A|0.83,1)\mathcal{N}(\theta_B|0.37,1)$. </li>
<span style="color:blue"> 
The performance depends on the initial conditions.In general, it seems that using the prior distribution slightly improves the results.
</span>
</ol>	 
</ul>    